In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import spearmanr, pearsonr
import seaborn as sns
from sklearn.decomposition import PCA
import umap
import hdbscan
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import shap
import random
import itertools
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

assays = ['DNase', 'H3K36me3', 'H3K27me3', 'H3K27ac', 'H3K4me1', 'H3K4me3', 'H3K9me3', 'Methylation_pos', 'Methylation_neg', 'MNase']
assay_index_choice = 10 # MNase

RESOLUTION = 25
window_size = 401
operative_half_window_size = 80 # will lead to a total of the central 2*operative_half_window_size + 1 features being chosen
pairwise_flag = True # If true then H3K27ac features will be concatentated with assays[assay_index_choice-1]
if(pairwise_flag):
    pairwise_multiplier = 0
else:
    pairwise_multiplier = 1


In [2]:
colnames = ["pos_" + str(x) for x in list(range(-(window_size//2), (window_size//2) + 1))]
colnames = colnames + ["log10p1(TPM)", "cell_type", "chrom", "position", "strand", "assay_index"] 
df = pd.read_csv('../Data/MNase.Training_Data.csv', sep=",", names=colnames, low_memory=False)
df_sorted = df.sort_values(['chrom', 'position', 'cell_type', 'assay_index'], ascending=[1, 1, 1, 1])
df_sorted_unique = df_sorted.drop_duplicates()
df_H3K27ac = df_sorted_unique.iloc[range(4-1, len(df_sorted_unique), len(assays)), :]
df_other_assay = df_sorted_unique.iloc[range(assay_index_choice-1, len(df_sorted_unique), len(assays)), :]
df_merged = pd.merge(df_H3K27ac, df_other_assay, how='inner', on=['cell_type', 'chrom', 'position', 'strand'], suffixes=['_H3K27ac', '_'+assays[assay_index_choice-1]])


In [3]:
middle_position_first_feature = window_size // 2
middle_position_second_feature = -1 + ( (window_size + 6) + (window_size + 6 + window_size + 1) )//2

df_two_assays = df_merged.iloc[:, 
                               np.r_[middle_position_first_feature-(operative_half_window_size):
                                     middle_position_first_feature+(operative_half_window_size)+1, 
                                     middle_position_second_feature-(operative_half_window_size):
                                     middle_position_second_feature+(operative_half_window_size)+1,
                                     window_size,
                                     window_size + 1,
                                     window_size + 2,
                                     window_size + 3]]

df_subsampled = df_two_assays

# Train on all but CXCR4 and TGFBR1 chromosomes
odd_chroms = ["chr" + str(c) for c in range(1, 23, 1)]
odd_chroms.remove("chr2") # exclude chr2 where CXCR4 lies
odd_chroms.remove("chr9") # exclude chr2 where TGFBR1 lies

df_odd_chroms = df_subsampled.loc[df_subsampled['chrom'].isin(odd_chroms)]
xTrain = df_odd_chroms.iloc[:, pairwise_multiplier*(2 * operative_half_window_size + 1):2 * (2 * operative_half_window_size + 1)] 
yTrain = df_odd_chroms.iloc[:, 2 * (2 * operative_half_window_size + 1)]

even_chroms = ["chr" + str(c) for c in range(2, 23, 2)]
df_even_chroms = df_subsampled.loc[df_subsampled['chrom'].isin(even_chroms)]
xTest = df_even_chroms.iloc[:, pairwise_multiplier*(2 * operative_half_window_size + 1):2 * (2 * operative_half_window_size + 1)] 
yTest = df_even_chroms.iloc[:, 2 * (2 * operative_half_window_size + 1)]

if(len(xTrain) < 10):
    print("len(xTrain) = "+str(len(xTrain)))
    continue

quadratic_transform = PolynomialFeatures(degree=1, interaction_only=True).fit(xTrain)
xTrain = quadratic_transform.transform(xTrain)
xTest = quadratic_transform.transform(xTest)

print(xTrain.shape, xTest.shape, yTrain.shape, yTest.shape)

# Fit a ridge regression model
l1_alpha = 0.001
model = linear_model.ElasticNet(alpha=l1_alpha, l1_ratio=0.0, max_iter=1000)

xTrain = np.asarray(xTrain)
yTrain = np.asarray(yTrain)
xTest = np.asarray(xTest)
yTest = np.asarray(yTest)

model.fit(xTrain, yTrain)

# Compute statistics
yTrain_Pred = model.predict(xTrain)
yTest_Pred = model.predict(xTest)
mse_Train = mean_squared_error(yTrain, yTrain_Pred)
pc_Train, _ = pearsonr(yTrain, yTrain_Pred)
sc_Train, _ = spearmanr(yTrain, yTrain_Pred)
mse_Test = mean_squared_error(yTest, yTest_Pred)
pc_Test, _ = pearsonr(yTest, yTest_Pred)
sc_Test, _ = spearmanr(yTest, yTest_Pred)

print("Train: MSE = ", round(mse_Train, 3), "Pearson =", round(pc_Train, 3), "Spearman =", round(sc_Train, 3))
print("Test: MSE = ", round(mse_Test, 3), "Pearson =", round(pc_Test, 3), "Spearman =", round(sc_Test, 3))

SyntaxError: 'continue' not properly in loop (<ipython-input-3-38f6727b59d6>, line 32)

In [ ]:
inserted_pvalue_choice = 1.5
peak_width_choice = {}
peak_width_choice["CXCR4"] = 6
peak_width_choice["TGFBR1"] = 6
assay_index = assay_index_choice

# Perform inference for epigenome editing data
def ise(gene, trained_model, assay_index, inserted_lnp1_minuslog10p_value = 3, peak_width = 2, pairwise_features=False):

    X = np.load("../Data/" + gene + ".npy")

    # obtain the middle portion of this
    X = X[:, (window_size//2)-operative_half_window_size:(window_size//2)+operative_half_window_size+1, :]

    # Perform inference by introducing p-value changes with a peak width
    yPred = []
    center = operative_half_window_size
    positions = range(center - center, center + center + 1)
    for pos in positions:
        X_modified = np.copy(X)
        for p in range(pos - peak_width // 2, pos + peak_width // 2 + 1):
            if( (p>=0) and (p < max(positions)) ):
                if(X_modified[:, p, 2] > 10): # Remember this is ln( -log10(p-value) + 1)
                    # If H3K27me3 peak exists, then p300 doesn't work
                    print("H3K27me3 exists!")
                    pass
                else:
                    if (pairwise_features):
                        # Modify the H3K27ac peak                                                
                        X_modified[:, p, 3] = X_modified[:, p, 3] + (X_modified[:, p, 9] * inserted_lnp1_minuslog10p_value)
                    else:
                        # Modify the assay itself
                        X_modified[:, p, assay_index] += inserted_lnp1_minuslog10p_value

        # Prepare input
        if (pairwise_features):
            X_modified = np.concatenate([X_modified[:, :, 3], X_modified[:, :, assay_index]], axis=1)
        else:                 
            X_modified = X_modified[:, :, assay_index]

        yPred_value = trained_model.predict(quadratic_transform.transform(X_modified))
        yy = yPred_value[0]
        yPred.append(yy)

    # Prepare input for predicting native expression
    if(pairwise_features):
        X = np.concatenate([X[:, :, 3], X[:, :, assay_index]], axis=1) 
    else:
        X = X[:, :, assay_index]

    # Instead of scaling, divide by yPred
    yPred_value = trained_model.predict(quadratic_transform.transform(X))[0] + 0.00000001 # to avoid divby0
    print("Predicted TPM for ", gene, " = ", (np.power(10, yPred_value) -1))
    yPred = (np.power(10, yPred) -1) / (np.power(10, yPred_value) -1)

    return yPred


def p_value_mapping(inserted_lnp1_minuslog10p_value):
    minuslog10p_value = np.expm1(inserted_lnp1_minuslog10p_value)
    p_value = np.power(10, -1 * minuslog10p_value)
    return round(minuslog10p_value, 4)


def convert_to_2D(idx, nrows, ncols):
    return idx//ncols, idx%ncols


# Load p300 epigenome editing data
df_p300_epigenome_editing = pd.read_csv("../Data/p300_epigenome_editing_dataset.tsv", sep="\t")

TSS = {}
STRANDS = {}
CHROMS = {}
GENES = {}
gRNA_STRANDS = {}
for index in range(len(df_p300_epigenome_editing)):
    tss = df_p300_epigenome_editing.iloc[index, 13]
    gene_strand = df_p300_epigenome_editing.iloc[index, 3]
    chrom = df_p300_epigenome_editing.iloc[index, 2]
    gene = df_p300_epigenome_editing.iloc[index, 0]

    TSS[gene] = int(tss)

    if(gene_strand == "plus"):
        STRANDS[gene] = "+"
    elif(gene_strand == "minus"):
        STRANDS[gene] = "-"
    else:
        print("something wrong with strand!")

    CHROMS[gene] = chrom
    GENES[gene] = 1                                                    

GENES_LIST = set(list(GENES.keys()))    

df_GENES_values = {}
df_GENES_means = {}
for gene in GENES_LIST:
    df_GENES_values[gene] = df_p300_epigenome_editing[df_p300_epigenome_editing["p300 target gene"] == gene]
    df_GENES_values[gene]["Position_wrt_TSS"] = pd.to_numeric(df_GENES_values[gene]["gRNA position  wrt TSS (hg38)"], errors='coerce')/RESOLUTION
    df_GENES_values[gene]["gRNA_strand"] = df_GENES_values[gene]["gRNA target strand"].apply(lambda s: "red" if(s=="minus") else "blue")
    df_GENES_values[gene]["gRNA_strand_bool"] = df_GENES_values[gene]["gRNA target strand"].apply(lambda s: -1 if(s=="minus") else +1)

    df_GENES_means[gene] = df_GENES_values[gene].groupby('Position_wrt_TSS').mean()
    df_GENES_means[gene].index.name = 'Position_wrt_TSS'
    df_GENES_means[gene].reset_index(inplace=True)

# Perform in-silico epigenesis
assay_color = ['black', 'red', 'green', 'blue', 'cyan', 'pink', 'brown']
xticklabels = range(-operative_half_window_size, operative_half_window_size + 1)

GENES_LIST = ["CXCR4", "TGFBR1"]

fig, axes = plt.subplots(nrows=len(GENES_LIST), ncols=2, figsize=(40, 30), sharey=False)
fig.tight_layout(pad=1, w_pad=20, h_pad=25)

for idx, gene in enumerate(sorted(GENES_LIST)):

    idx_x, idx_y = convert_to_2D(idx, nrows=len(GENES_LIST), ncols=1)
    ax_1 = axes[idx_x, 0]
    ax_2 = axes[idx_x, 1]

    gene_features = np.squeeze(np.load("../Data/" + gene + ".npy"), axis=0)
    gene_features = gene_features[(window_size//2)-operative_half_window_size:(window_size//2)+operative_half_window_size+1, :]

    df_values = df_GENES_values[gene]
    df_means = df_GENES_means[gene]

    inserted_lnp1_minuslog10p_value = inserted_pvalue_choice
    peak_width = peak_width_choice[gene]
    gene_ise = ise(gene, model, assay_index-1, inserted_lnp1_minuslog10p_value, peak_width, pairwise_flag)

    # Create a scatter plot of the means with the predictions of those positions
    gene_ise_at_means = []
    alan_means = []
    gRNA_strands = []

    for p_idx in range(len(df_means)):
        position_m = df_means.iloc[p_idx, 0]
        alan_mean = df_means.iloc[p_idx, 1]
        gRNA_strand = df_means.iloc[p_idx, 9]
        if(position_m + operative_half_window_size < 0):
            continue
        elif(position_m > operative_half_window_size):
            continue
        else:
            gene_ise_at_means.append(gene_ise[int(position_m) + operative_half_window_size])
            alan_means.append(alan_mean)
            gRNA_strands.append(gRNA_strand)

    pc, pp = pearsonr(list(alan_means), gene_ise_at_means)
    sc, sp = spearmanr(list(alan_means), gene_ise_at_means)

    gRNA_strands_colors = []
    for iii in gRNA_strands:
        if(iii == -1):
            gRNA_strands_colors.append("red")
        else:
            gRNA_strands_colors.append("blue")

    ax_1.scatter(list(alan_means), gene_ise_at_means, color="#FF1493", s=1000) # ="#FF1493")
    ax_1.set_xlim(0, 1.1 * max(alan_means))
    ax_1.set_ylim(0, 1.1 * max(gene_ise_at_means))
    ax_1.tick_params(axis='both', which='major', labelsize=40)
    ax_1.tick_params(axis='both', which='minor', labelsize=40)
    ax_1.set_xlabel("Mean experimental fold change", size=60)
    ax_1.set_ylabel("Model prediction's fold change", size=45)
#     if(pp > 0.05):
#         p_asterisk = "NS"
#     if(sp > 0.05):
#         s_asterisk = "NS"
    ax_1.set_title("Correlation between experimental and model predictions fold change\nPearson = "+
                   str(round(pc, 2))+
                   " ("+str(round(pp, 3))+
                   ") Spearman = "+
                   str(round(sc, 2))+
                   " ("+str(round(sp, 3))+
                   ")", size=40)

    # Determine whether we are doing H3K27ac ISE in the background of another assay's features
    # or we have marginal features and are doing that track's ISE
    if(pairwise_flag):
        epigenetic_features = gene_features[:, 9] # MNase
        epigenetic_features_2 = gene_features[:, 3] # H3K27ac
        color_for_assay = assay_color[3]
        label_for_assay = assays[3]
    else:
        epigenetic_features = gene_features[:, assay_index-1]
        color_for_assay = assay_color[assay_index-1]
        label_for_assay = assays[assay_index-1]

    # Scale the model predictions     
    scaling_ratio = np.median(df_means['Measured fold change'])/np.median(gene_ise - 0.0)
    scaled_model_predictions = 0.2 * (scaling_ratio * (gene_ise - 0.0)) + 1.0

    # Scale the epigenetic features
    epigenetic_features_scaling_ratio = max(df_means['Measured fold change'])/max(epigenetic_features - 0.0)
    scaled_epigenetic_features = (epigenetic_features_scaling_ratio * (epigenetic_features - 0.0)) + 0.5

    scaled_epigenetic_features_2 = (epigenetic_features_scaling_ratio * (epigenetic_features_2 - 0.0)) + 0.5 

    ax_2.plot(xticklabels, scaled_model_predictions, 'o-', color="#4daf4a", linewidth=5, markersize=2, label="(Scaled) Model Predictions " + label_for_assay)
    ax_2.plot(xticklabels, scaled_epigenetic_features, 'o-', color="#8470FF", linewidth=5, markersize=1, label="(Scaled) Epigenetic Features MNase") # + label_for_assay)
    ax_2.plot(xticklabels, scaled_epigenetic_features_2, 'o-', color="darkblue", linewidth=5, markersize=1, label="(Scaled) Epigenetic Features H3K27ac") # + label_for_assay)

    ax_2.bar(df_means['Position_wrt_TSS'], 0.0 + (df_means['Measured fold change']), color="#f781bf", bottom=0, width=2, label="Experimental mean from qPCR")

    gRNA_strand_groups = df_values.groupby("gRNA target strand")

    color_dict = {}
    color_dict["+"] = {"plus":"blue", "minus":"red"}
    color_dict["-"] = {"plus":"red", "minus":"blue"}
    direction = {"blue":"rightward", "red":"leftward"}
    color_index = 0
    for name, group in gRNA_strand_groups:
        ax_2.plot(group['Position_wrt_TSS'], 0.0 + (group['Measured fold change']), 'o', color=color_dict[STRANDS[gene]][name], label="qPCR facing "+direction[color_dict[STRANDS[gene]][name]], markersize=15)
        color_index += 1

    ax_2.set_xlim(-operative_half_window_size-10, operative_half_window_size+10)
    ax_2.set_ylim(-1, 1.0 + max(df_means['Measured fold change'])*1.5)
    x_vals = ax_2.get_xticks()
    ax_2.set_xticklabels(['{:3.0f}'.format(x * RESOLUTION) for x in x_vals])
    ax_2.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax_2.tick_params(axis='both', which='major', labelsize=35)
    ax_2.tick_params(axis='both', which='minor', labelsize=35)
    ax_2.set_xlabel("Peak Position (in bp) w.r.t TSS", size=50)
    ax_2.set_ylabel("Gene expression fold change", size=50)
    ax_2.set_title(gene+" with H3K27ac + "+assays[assay_index-1]+"\nincreasing " + str(peak_width * RESOLUTION) + "bp peaks by -log10(p_value)="+str(p_value_mapping(inserted_lnp1_minuslog10p_value)), size=40) #, y=1.1)

    ax_2.legend(loc='upper center', prop={'size': 30}, ncol=2)

plt.show()
plt.close()